<h1>Test db_connect.py</h1>

In [1]:
import sys
import os
dir_path = os.getcwd()
sys.path.insert(0,dir_path)
print(sys.path)

['/home/kiryl/Documents/Projects/Website_Parser', '/home/kiryl/Documents/Environments/scraper/lib/python36.zip', '/home/kiryl/Documents/Environments/scraper/lib/python3.6', '/home/kiryl/Documents/Environments/scraper/lib/python3.6/lib-dynload', '/usr/lib/python3.6', '', '/home/kiryl/Documents/Environments/scraper/lib/python3.6/site-packages', '/home/kiryl/Documents/Environments/scraper/lib/python3.6/site-packages/IPython/extensions', '/home/kiryl/.ipython']


In [2]:
from parse import simple_get, get_soup, get_product_parameters
from parse import parse_product_info_into_db

from db_connect import *

<h3>Test simple db connection and retrieve operation</h3>

In [3]:
#instantiate database connection using Posgres_db class from db_connect.py
db_connection = Postgres_db()
result = db_connection.query("SELECT * FROM products_all WHERE id=8779")
for entry in result:  
    print(f'\n id: {entry.id}')
    print(f'\n url: {entry.url}')
    print(f'\n category: {entry.category}')
    print(f'\n subcategory lvl 1: {entry.subcategory_lvl1}')
    print(f'\n subcategory lvl 2: {entry.subcategory_lvl2}')
    

products_all = db_connection.reflect_table('products_all')


 id: 8779

 url: https://www.oma.by/ugolok-d-plitki-pvkh-11-12-mm-naruzhnyy-2-5m-belyy-2-260638-p

 category: Отделка

 subcategory lvl 1: Плитка

 subcategory lvl 2: Аксессуары для плитки


/home/kiryl/Documents/Environments/scraper/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


<h3>Test table reflection</h3>

In [4]:
db = Postgres_db()
initial_select_string = 'SELECT * FROM products_all WHERE is_parsed IS NULL LIMIT 2'
entries_to_parse = db.query(initial_select_string)
products_all = db.reflect_table('products_all')

for entry in entries_to_parse: 
    response = simple_get(entry.url)
    soup = get_soup(response)
    p_dict = get_product_parameters(soup)
    print(f'\n\n{p_dict}')
    dt = datetime.now()
    if entry.is_parsed == None:
        update_sttm = products_all.update().where(products_all.c.id==entry.id).values(product_name = p_dict['product_name'], \
                                                                                      product_price = float(p_dict['product_price']), \
                                                                                      product_description = p_dict['product_description'], \
                                                                                      product_characteristics = p_dict['product_characteristics'], \
                                                                                      similar_products = p_dict['similar_products'], \
                                                                                      product_image_link = p_dict['product_image_link'], \
                                                                                      product_is_hit = p_dict['product_is_trend'], \
                                                                                      is_parsed = dt)                                                                                                    
        db.query(update_sttm)
        print(f'{entry.id} is parsed')




{'product_name': 'Плитка для ванной Керамин Мишель 7С панно тип 2 декор 400 x 275 мм, Минск ', 'product_price': 5.8, 'product_description': '\n                                    Плитка для ванной Керамин Мишель 7С панно тип 2 предназначена для облицовки стен в ванной. Изготовлена из керамики. Добавляет интерьеру эффектный и органичный вид. Плитка обладает стойкостью к истиранию, перепадам температур, воздействию чистящих средств. Подходит для декора помещений в любом стиле. Цвет - зеленый. Размеры - 400 x 275 мм.', 'product_characteristics': 'Бренд товараБрендКераминОсновные характеристикиМатериалкерамикаВиддекорНазначениедля ванной комнатыдля кухниТип поверхностиматоваяЦветбежевыйКоллекцияМишель/MishelДля отделкивнутри помещенийРазмеры и количествоДлина плитки, мм400Ширина плитки, мм275Площадь одной плитки, м20.11Количество штук в 1 м29.091Количество в упаковке, шт13                                    Все характеристики', 'similar_products': '', 'product_image_link': 'https://www.o

<h3>Test <i>parse_product_info_into_db</i> fuction from parse.py module</h3>

In [9]:
%%time
parsed_count = parse_product_info_into_db(number_of_entries_to_parse = 20)
print(f'number of parsed items: {parsed_count}\n')

number of parsed items: 20

CPU times: user 9.91 s, sys: 35.9 ms, total: 9.94 s
Wall time: 24.3 s
